In [1]:
#Read Data From CSV file
import pandas as pd
df=pd.read_csv('NEWDATATABLE.csv', sep=',',header=None)
ratings = df.values
import numpy as np
import matplotlib.pyplot as plt

In [199]:
def train_test_generation(ratings):
    #Cretae train and test matrix
    test = np.ones(ratings.shape) * 99 #24983*100 and initzalize all be unknown 99 and ratings.shape[0] return 24983
    train = ratings.copy()
    #Make all unknown numbers 99 be 0
    train = train - np.ones(ratings.shape) * 99
    for user in xrange(ratings.shape[0]):
        #Randomly pick 10 ratings from each user to be in test case (include unknown ratings)
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size = 20, replace = False)
        #Set chosen be zero in training
        train [user, test_ratings] = 0.
        #Assign chosen rating to test case
        test[user, test_ratings] = ratings[user, test_ratings]
    
    train = train + np.ones(ratings.shape) * 99
    #Make sure train and test are disjoint 
    #assert(np.all((train * test) == 0))
    return train, test

In [200]:
#Call the function and then assign train and test
train, test = train_test_generation(ratings)
trainMAE = train.copy()

In [201]:
#Find averge of each column
train_average_column = np.zeros(100)
for indexI in range(100):
    tempsum = 0
    count = 0
    for indexJ in range(24983):
        if train[indexJ][indexI] != 99:
            tempsum = tempsum + train[indexJ][indexI]
            count = count + 1
    train_average_column[indexI] = float(tempsum) / count
#Find averge of each row
train_average_row = np.zeros(24983)
for indexI in range(24983):
    tempsum = 0;
    count = 0; 
    for indexJ in range(100):
        if train[indexI][indexJ] != 99:
            tempsum = tempsum + train[indexI][indexJ]
            count = count + 1
    train_average_row[indexI] = float(tempsum) / count

In [202]:
#train_norm is used for svd and train is used for CF item/user based
train_norm = train.copy()
#Fill the sparse with avergae of item score
for indexI in range(24983):
    for indexJ in range(100):
        if train_norm[indexI][indexJ] == 99:
            train_norm[indexI][indexJ] = train_average_column[indexJ]

In [203]:
#Substraction of customer AVG from each rating
for indexI in range(24983):
    for indexJ in range(100):
        train_norm[indexI][indexJ] = train_norm[indexI][indexJ] - train_average_row[indexI]

In [204]:
#from sklearn.metrics import mean_absolute_error
#Compute MAE in training and test
def get_mae(pred, actual):
    mae_sum = 0
    count = 0
    for indexI in range(24983):
        for indexJ in range(100):
            if actual[indexI][indexJ] != 99:
                mae_sum = mae_sum + np.abs(pred[indexI][indexJ] - actual[indexI][indexJ])
                count = count + 1    
    mae_sum = float(mae_sum) / count
    return mae_sum

In [205]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(train_norm, k=10)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + train_average_row.reshape(-1,1)

print 'SVD training MAE: ' + str(get_mae(all_user_predicted_ratings, train))
print 'SVD testing MAE: ' + str(get_mae(all_user_predicted_ratings, test))

SVD training MAE: 2.8567441853
SVD testing MAE: 3.28782178729


In [236]:
basis = 60
train_first_half = train[:,0:basis]
train_second_half = train[:,basis:]

In [237]:
#Compute train_first_half
#Find averge of each column 
train_average_column_first_half = np.zeros(basis)
for indexI in range(basis):
    tempsum = 0
    count = 0
    for indexJ in range(len(train_first_half)):
        if train_first_half[indexJ][indexI] != 99:
            tempsum = tempsum + train_first_half[indexJ][indexI]
            count = count + 1
    if count != 0:
        train_average_column_first_half[indexI] = float(tempsum) / count
    else:
        break
#Find averge of each row
train_average_row_first_half = np.zeros(len(train_first_half))
for indexI in range(len(train_first_half)):
    tempsum = 0;
    count = 0; 
    for indexJ in range(basis):
        if train_first_half[indexI][indexJ] != 99:
            tempsum = tempsum + train_first_half[indexI][indexJ]
            count = count + 1
    if count != 0:
        train_average_row_first_half[indexI] = float(tempsum) / count
    else:
        break

In [238]:
#train_norm is used for svd and train is used for CF item/user based
train_norm_first_half = train_first_half.copy()
#Fill the sparse with avergae of item score
for indexI in range(len(train_first_half)):
    for indexJ in range(basis):
        if train_norm_first_half[indexI][indexJ] == 99:
            train_norm_first_half[indexI][indexJ] = train_average_column_first_half[indexJ]

#Substraction of customer AVG from each rating
for indexI in range(len(train_first_half)):
    for indexJ in range(basis):
        train_norm_first_half[indexI][indexJ] = train_norm_first_half[indexI][indexJ] - train_average_row_first_half[indexI]

In [239]:
#Compute train_second_half
#Find averge of each column 
train_average_column_second_half = np.zeros(100-basis)
for indexI in range(100-basis):
    tempsum = 0
    count = 0
    for indexJ in range(len(train_second_half)):
        if train_second_half[indexJ][indexI] != 99:
            tempsum = tempsum + train_second_half[indexJ][indexI]
            count = count + 1
    if count != 0:
        train_average_column_second_half[indexI] = float(tempsum) / count
    else:
        break
#Find averge of each row
train_average_row_second_half = np.zeros(len(train_second_half))
for indexI in range(len(train_second_half)):
    tempsum = 0;
    count = 0; 
    for indexJ in range(100-basis):
        if train_second_half[indexI][indexJ] != 99:
            tempsum = tempsum + train_second_half[indexI][indexJ]
            count = count + 1
    if count != 0:
        train_average_row_second_half[indexI] = float(tempsum) / count
    else:
        break

In [240]:
#train_norm is used for svd and train is used for CF item/user based
train_norm_second_half = train_second_half.copy()
#Fill the sparse with avergae of item score
for indexI in range(len(train_second_half)):
    for indexJ in range(100-basis):
        if train_norm_second_half[indexI][indexJ] == 99:
            train_norm_second_half[indexI][indexJ] = train_average_column_second_half[indexJ]

#Substraction of customer AVG from each rating
for indexI in range(len(train_second_half)):
    for indexJ in range(100-basis):
        train_norm_second_half[indexI][indexJ] = train_norm_second_half[indexI][indexJ] - train_average_row_second_half[indexI]

In [241]:
#Compute based SVD
U, sigma, Vt = svds(train_norm_first_half, k=10)
sigma = np.diag(sigma)

In [242]:
#Compute to find projection of terms on to span of current documents
from numpy.linalg import inv
sigma_inv = inv(sigma)
d_hut = np.dot(np.dot(train_norm_second_half.T,U),sigma_inv)

In [243]:
combined = np.concatenate((Vt, d_hut.T), axis=1)
all_user_predicted_ratings_final = np.dot(np.dot(U, sigma), combined)

In [244]:
combined_avg = train_average_row
all_user_predicted_ratings_final_withavg = all_user_predicted_ratings_final + combined_avg.reshape(-1,1)

In [245]:
print 'SVD training MAE: ' + str(get_mae(all_user_predicted_ratings_final_withavg, train))
print 'SVD testing MAE: ' + str(get_mae(all_user_predicted_ratings_final_withavg, test))

SVD training MAE: 2.92424312306
SVD testing MAE: 3.38386505253
